In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from helpers import *
import numpy as np

In [3]:
data = [ 
    'data/liquor.json',
    'data/social_cocktail.json',
    'data/serious_eats.json',
    'data/live_in_style.json',
    'data/all_recipes.json'
]

descriptions, ingredients, names = [], [], []

for d in data:
    descriptions += load_data(d, field='description')
    ingredients += load_data(d, field='ingredients')
    names += load_data(d, field='name')

assert len(descriptions) == len(ingredients)

recipes = [x + ' # ' + y for x, y in zip(ingredients, descriptions)]    
print('There are {} recipes in the database.'.format(len(recipes)))

There are 3347 recipes in the database.


In [4]:
T = [clean_string(x) for x in ingredients]
T = sentences_to_words(T)

N = [clean_string(x) for x in names]
N = [x.split() for x in N]

rIngr = []
rNames = []


for i,t in enumerate(T):
    if len(t) <= 30 and len(N[i]) <= 8:
        rIngr = rIngr + [t]
        rNames = rNames + [N[i]]
    
rIngr = [np.append(x, ["null"]*(30-len(x))) for x in rIngr]
rNames = [np.append(x, ["null"]*(8-len(x))) for x in rNames]

rIngr = np.array(rIngr)
rNames = np.array(rNames)

In [5]:
print(rIngr.shape)
print(rNames.shape)

(3292, 30)
(3292, 8)


In [6]:
print(rIngr[123])
print(rNames[123])

['fresh' 'juice' 'simple' 'leaves' 'lemon' 'syrup' 'null' 'null' 'null'
 'null' 'null' 'null' 'null' 'null' 'null' 'null' 'null' 'null' 'null'
 'null' 'null' 'null' 'null' 'null' 'null' 'null' 'null' 'null' 'null'
 'null']
['south' 'side' 'null' 'null' 'null' 'null' 'null' 'null']


In [7]:
label_encoder = LabelEncoder()
flat = np.append(rNames.flatten(),rIngr.flatten())
label_encoder.fit(flat)
k = label_encoder.transform([[x] for x in list(set(flat))])
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.fit(k.reshape(-1,1))


fIngr = []
fNames = []

for i,_ in enumerate(rNames):
    fIngr += [np.array(onehot_encoder.transform(label_encoder.transform(rIngr[i]).reshape(-1,1))).flatten()]
    fNames += [np.array(onehot_encoder.transform(label_encoder.transform(rNames[i]).reshape(-1,1))).flatten()]
    
X = np.array(fIngr)
y = np.array(fNames)

C:\Users\Martin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
N = X.shape[0]
X_train, y_train = X[:int(N*0.95)], y[:int(N*0.95)]
X_val, y_val = X[int(N*0.95):], y[int(N*0.95):]
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(3127, 124560) (3127, 33216) (165, 124560) (165, 33216)


In [9]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [10]:
model = Sequential()
model.add(Dense(512, input_dim=X.shape[1], activation='relu'))
# model.add(Dense(256, input_dim=X.shape[1], activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))

# model.add(GRU(512, input_shape=(X.shape[1], X.shape[2]), activation='relu'))
# model.add(Dropout(0.2))
# model.add(GRU(128, input_shape=(X.shape[1], X.shape[2]), activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(33216, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'], optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               63775232  
_________________________________________________________________
dense_2 (Dense)              (None, 33216)             17039808  
Total params: 80,815,040
Trainable params: 80,815,040
Non-trainable params: 0
_________________________________________________________________
None


In [55]:
batch_size = 32
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=batch_size)

Train on 3127 samples, validate on 165 samples
Epoch 1/10
3127/3127 [==============================] - 229s 73ms/step - loss: 17.7178 - categorical_accuracy: 0.1976 - val_loss: 45.9632 - val_categorical_accuracy: 0.0000e+00
Epoch 2/10
3127/3127 [==============================] - 245s 78ms/step - loss: 17.6572 - categorical_accuracy: 0.2069 - val_loss: 45.9557 - val_categorical_accuracy: 0.0000e+00
Epoch 3/10
3127/3127 [==============================] - 241s 77ms/step - loss: 17.5902 - categorical_accuracy: 0.2143 - val_loss: 45.8997 - val_categorical_accuracy: 0.0000e+00
Epoch 4/10
3127/3127 [==============================] - 251s 80ms/step - loss: 17.5475 - categorical_accuracy: 0.2165 - val_loss: 46.2545 - val_categorical_accuracy: 0.0000e+00
Epoch 5/10
3127/3127 [==============================] - 269s 86ms/step - loss: 17.5171 - categorical_accuracy: 0.2245 - val_loss: 46.1039 - val_categorical_accuracy: 0.0000e+00
Epoch 6/10
3127/3127 [==============================] - 303s 97ms/st

In [66]:
ent = 100

predict = X_val[ent]
p = model.predict(x=np.array([predict]),batch_size=batch_size)
p = p.reshape(8,4152)
b = np.zeros_like(p)
b[np.arange(len(p)), p.argmax(1)] = 1
decoded = b.dot(onehot_encoder.active_features_).astype(int)
result = label_encoder.inverse_transform(decoded)


print(rIngr[ent])
print(rNames[ent])
print(result)


['vodka' 'blanc' 'plymouth' 'absolut' 'lillet' 'null' 'null' 'null' 'null'
 'null' 'null' 'null' 'null' 'null' 'null' 'null' 'null' 'null' 'null'
 'null' 'null' 'null' 'null' 'null' 'null' 'null' 'null' 'null' 'null'
 'null']
['vesper' 'null' 'null' 'null' 'null' 'null' 'null' 'null']
['blue' 'martini' 'null' 'null' 'null' 'null' 'null' 'null']


In [ ]:
model_file = 'm30.json'
weights_file = 'w30.h5'

def save_model_to_json(model, model_file, weights_file):
    model_json = model.to_json()
    with open(model_file, "w") as json_file:
        json_file.write(model_json)
    model.save_weights(weights_file)
    
save_model_to_json(model, model_file, weights_file)